<a href="https://colab.research.google.com/github/BrMrtn/GoogleColab/blob/main/K%C3%A9pfeldolgoz%C3%A1s/ip_practice06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go

# Image Processing Practice 6

1. Pinhole cameras
2. 3D point cloud reconstruction
3. The robustness of stereo matching
4. Normal vector reconstruction
5. Sources

-----------------------

# 1. Pinhole cameras

## 1. 1. Linear algebra reminder

Inverse of an $A \in \mathbb{R}^{3×3}$ matrix:

* notation: $A^{-1}$
* property: $A^{-1}A=\begin{bmatrix} 1&0&0 \\ 0&1&0 \\ 0&0&1 \end{bmatrix}$

Matrix-vector multiplication:

$$\begin{bmatrix} a_{11}&a_{12}&a_{13} \\ a_{21}&a_{22}&a_{23} \\ a_{31}&a_{32}&a_{33} \end{bmatrix} \begin{bmatrix} v_1 \\ v_2 \\ v_3 \end{bmatrix}= v_1 \begin{bmatrix} a_{11} \\ a_{21} \\ a_{31} \end{bmatrix} + v_2 \begin{bmatrix} a_{12} \\ a_{22} \\ a_{32} \end{bmatrix} + v_3 \begin{bmatrix} a_{13} \\ a_{23} \\ a_{33} \end{bmatrix}$$

## 1.2. How to get the pixel corresponding to a point in the space

**Simplifying assumption:** Let the $(0, 0)$ pixel the bottom left corner.

Figure:

<svg height="260" width="300">
  <text x="60" y="20" font-weight="bold">Image coordinates</text>
  <rect width="200" height="200" fill="none" stroke="black" y="40" x="25"/>
  <circle cx="25" cy="240" r="10" fill="red"/>
  <text x="0" y="220">(0, 0)=bottom left corner</text>
  <text x="120" y="250">x_px</text>
  <text x="0" y="150">y_px</text>
</svg>

Image space:

* horizontally flipped version of the pixels
* bottom left corner: $(0, 0)$

Given:

* A pinhole camera (e. g. smartphone camera). The camera has the following properties:
  * Looks at the +Z-direction
  * The up-direction is Y
  * Its intrinsic matrix is called K
* A point in front of the camera. Relative position to the camera: $\begin{bmatrix} x_{\text{space}} \\ y_{\text{space}} \\ z_{\text{space}} \end{bmatrix}$

Figure:

<svg height="200" width="600">
    <line x1="0" y1="100" x2="100" y2="50" stroke="black"/>
    <!-- Y axis -->
    <text x="10" y="20">Y</text>
    <line x1="0" y1="100" x2="0" y2="0" stroke="black"/>
    <!-- Z axis -->
    <text x="40" y="90">Z</text>
    <line x1="0" y1="100" x2="100" y2="200" stroke="black"/>
    <!-- X axis -->
    <text x="40" y="160">X</text>
    <!-- Relevant point -->
    <circle cx="140" cy="70" r="5" fill="blue"/>
    <text x="140" y="70">(x_space, y_space, z_space)</text>
    <!--Camera -->
    <text x="0" y="100">Camera</text>
</svg>

The elements of the intrinsic matrix (reminder): $K=\begin{bmatrix} f_x&0&c_x \\ 0&f_y&c_y \\ 0&0&1 \end{bmatrix}$, where:

* $f_x, f_y$: horizontal and vertical focal lengths (they are typically equal)
* $c_x, c_y$: the optical center

The formula to get the pixel for an $(x_{\text{space}}, y_{\text{space}}, z_{\text{space}}$) point.

1. Calculate the pixel homogen coordinates.

$$\begin{bmatrix} x_{\text{h}} \\ y_{\text{h}} \\ z_{\text{space}} \end{bmatrix} = \underbrace{\begin{bmatrix} f_x&0&c_x \\ 0&f_y&c_y \\ 0&0&1 \end{bmatrix}}_{K} \begin{bmatrix} x_{\text{space}} \\ y_{\text{space}} \\ z_{\text{space}} \end{bmatrix}$$

2. Divide by the last coordinate.

$$ \begin{bmatrix} x_{\text{px}} \\ y_{\text{px}} \\ 1 \end{bmatrix} = \frac{1}{z_{\text{space}}} \begin{bmatrix} x_{\text{h}} \\ y_{\text{h}} \\ z_{\text{space}} \end{bmatrix}$$

In other words:

$$ x_{\text{px}} = \frac{f_x x_{\text{space}}+c_x z_{\text{space}}}{z_{\text{space}}} $$

$$ y_{\text{px}} = \frac{f_y y_{\text{space}}+c_y z_{\text{space}}}{z_{\text{space}}} $$

Python implementation:

In [ ]:
def get_im_pos(K: np.ndarray, pt_space: np.ndarray) -> np.ndarray:
    im_hom = K @ pt_space
    im_hom = im_hom / im_hom[-1]
    return im_hom[[0, 1]]

For example:

In [ ]:
K = np.array([
    [320, 0, 320],
    [0, 320, 240],
    [0, 0, 1]
], dtype=np.float32)
get_im_pos(K, np.array([
    [2],
    [1.5],
    [8]
]))

Now, let's multiply the pooint in the space, by 2. It falls onto the same pixel. Why is this unsurprising?

In [ ]:
get_im_pos(K, np.array([
    [2],
    [1.5],
    [8]
])*2)

Now, modify the X coordinate of the point while keeping the Y other coordinates constant. As you can see, the Y coordinate on the image does not change.

In [ ]:
get_im_pos(K, np.array([
    [2+3],
    [1.5],
    [8]
]))

## 1.3. Exercises

**Exercise 1:** Assume that we have two cameras with the same known intrinsics. They point at the +Z-direction. They have the same Y coordinate and have distance B. Assume that both see a point. What is the Z-coordinate of the point ($z_{\text{space}}$) relative to the cameras?

Figure:

<svg height="200" width="600">
    <!-- Y axis -->
    <line x1="0" y1="100" x2="100" y2="50" stroke="black"/>
    <text x="10" y="20">Y</text>
    <!-- Z axis -->
    <line x1="0" y1="100" x2="0" y2="0" stroke="black"/>
    <text x="40" y="90">Z</text>
    <!-- X axis -->
    <line x1="0" y1="100" x2="100" y2="200" stroke="black"/>
    <text x="40" y="160">X</text>
    <!-- Relevant point -->
    <circle cx="140" cy="70" r="5" fill="blue"/>
    <text x="140" y="70">(x_space, y_space, z_space)</text>
    <!--Left camera -->
    <text x="0" y="100">Left camera</text>
    <circle cx="0" cy="100" r="5" fill="red"/>
    <!--Right camera -->
    <text x="50" y="150">Right camera</text>
    <circle cx="50" cy="150" r="5" fill="red"/>
    <!-- B -->
    <line x1="0" y1="100" x2="50" y2="150" stroke="red"  stroke-dasharray="4"/>
    <text x="25" y="125" >B</text>
</svg>

Let:

* The x coordinate of the point relative to the **left** camera: $x_{\text{space}}$
* The x coordinate of the point relative to the **right** camera: $x'_{\text{space}}$
* the x coordinate of the pixel for the point on the image of the **left** camera: $x_{\text{px}}$
* the x coordinate of the pixel for the point on the image of **right** camera: $x'_{\text{px}}$
* The camera intrinsics: $K=\begin{bmatrix} f_x&0&c_x \\ 0&f_y&c_y \\ 0&0&1 \end{bmatrix}$


**Solution:** The formulas for the x coordinates of the pixels (copied from above):

$$x_{\text{px}} =\frac{f_x x_{\text{space}}+c_x z_{\text{space}}}{z_{\text{space}}}$$

$$x'_{\text{px}} =\frac{f_x x'_{\text{space}}+c_x z_{\text{space}}}{z_{\text{space}}}$$

Now, subtract the two equations:

$$x_{\text{px}}-x'_{\text{px}} = \frac{f_x \cdot (x_{\text{sapce}}-x'_{\text{sapce}})}{z_{\text{space}}}$$

Substitute $B=x_{\text{sapce}}-x'_{\text{sapce}}$

$$x_{\text{px}}-x'_{\text{px}} = \frac{f_x B}{z_{\text{space}}}$$

Reorganize the equation

$$z_{\text{space}} = \frac{f_x B}{x_{\text{px}}-x'_{\text{px}}}$$

**Important note:** $x_{\text{px}}-x'_{\text{px}}$ is called *disparity*, and it is proportional to $1/z_{\text{space}}$ *regardless of the camera intrinsics*.

**Exercise 2:** Assume that, there is a point and we know:

* It is at the $\begin{bmatrix} x_{\text{px}} \\ y_{\text{px}} \end{bmatrix}$ pixel.
* Its Z coordinate relative to the camera is $z_{\text{space}}$.
* The intrinsic matrix is K.

How to calculate its X and Y coordinates ($x_{\text{space}}, y_{\text{space}}$) relative to the camera?

**Solution:** Reorganize the original projection formula.

1. Get the on-image position in homogen coordinates:

$$z_{\text{space}} \begin{bmatrix}x_{\text{px}} \\ y_{\text{px}} \\ 1 \end{bmatrix}$$

2. Write the projection formula:

$$z_{\text{space}} \begin{bmatrix}x_{\text{px}} \\ y_{\text{px}} \\ 1 \end{bmatrix} = K\begin{bmatrix} x_{\text{space}} \\ y_{\text{space}} \\ z_{\text{space}} \end{bmatrix} $$

3. Multiply with $K^{-1}$


$$K^{-1} z_{\text{space}} \begin{bmatrix}x_{\text{px}} \\ y_{\text{px}} \\ 1 \end{bmatrix} = \begin{bmatrix} x_{\text{space}} \\ y_{\text{space}} \\ z_{\text{space}} \end{bmatrix}$$

--------------------

# 2. 3D reconstruction

Download the images

In [ ]:
!wget https://raw.githubusercontent.com/mntusr/tartanair-stereo-pair/refs/heads/main/tartan_000072_left.jpg
!wget https://raw.githubusercontent.com/mntusr/tartanair-stereo-pair/refs/heads/main/tartan_000072_right.jpg

Load the images

In [ ]:
# the image pair from the TartanAir-V1 dataset [1]
# (converted to jpg)
im_left = cv2.imread("tartan_000072_left.jpg")
im_left_rgb = cv2.cvtColor(im_left, cv2.COLOR_BGRA2RGB)
im_left_gray = cv2.cvtColor(im_left, cv2.COLOR_BGRA2GRAY)

im_right = cv2.imread("tartan_000072_right.jpg")
im_right_rgb = cv2.cvtColor(im_right, cv2.COLOR_BGRA2RGB)
im_right_gray = cv2.cvtColor(im_right, cv2.COLOR_BGRA2GRAY)

plt.subplot(1, 2, 1)
plt.imshow(im_left_rgb)
print(f"{im_right_rgb.shape=}")
plt.title("Left")
plt.subplot(1, 2,2)
plt.imshow(im_right_rgb)
print(f"{im_right_rgb.shape=}")
plt.title("Right");

Calculate the disparity value (from exercise 1) for each pixel, where possible

In [ ]:
# algorithm: stereo block matching
stereo = cv2.StereoBM.create()

# StereoBM assumes the configuration of exercise 1.
# Disparity calculation (simplified steps):
# 1. Take square blocks of one image
# 2. Look for them in the SAME ROW on the OTHER image (similarly to template matching)
# 3. Store the X position change.
# Why is no intrinsic matrix needed?
disparity = stereo.compute(im_left_gray, im_right_gray)

# show the disparity map
disp_out = plt.imshow(disparity)
plt.colorbar(disp_out);

Get the dimensions of the disparity map

In [ ]:
disp_height, disp_width = disparity.shape

Calculating the depth from the disparity

In [ ]:
depth = (100/disparity) # we want to visualize, so we do not care about scale

Calculate the homogen coordinates as a point cloud of homogen points.

In [ ]:
tartanair_camera = np.array([
    [320, 0, 320],
    [0, 320, 240],
    [0, 0, 1]
], dtype=np.float32)

def depth_2_point_cloud(depth: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    x, y = np.meshgrid(
        np.arange(0, disp_width),
        disp_height - np.arange(0, disp_height),
    )
    depth_flat = depth.flatten()
    x_im_flat = x.flatten()*depth_flat
    y_im_flat = y.flatten()*depth_flat

    x_im_flat[depth_flat<0] = np.nan
    y_im_flat[depth_flat<0] = np.nan
    depth_flat[depth_flat<0] = np.nan

    homogen_point_cloud = np.stack([x_im_flat, y_im_flat, depth_flat], axis=0).T
    # multiply each element of the point cloud with the inverse of the intrinsic matrix
    space_point_cloud = (np.linalg.inv(tartanair_camera) @ homogen_point_cloud.T).T
    # reduce the number of points
    point_indices = np.random.default_rng().choice(len(space_point_cloud), 40000)
    space_point_cloud = space_point_cloud[point_indices]

    # get the color for each point
    color_values = np.stack([
        im_left_rgb[:, :, 0].flatten(),
        im_left_rgb[:, :, 1].flatten(),
        im_left_rgb[:, :, 2].flatten(),
    ], axis=-1)
    color_values = color_values[point_indices]

    return space_point_cloud, color_values

space_point_cloud, color_values = depth_2_point_cloud(depth)

# we use plotly since its interactive mode works better in the browser
fig = go.Figure()
fig.add_trace(
    go.Scatter3d(
        x=space_point_cloud[:, 0],
        y=space_point_cloud[:, 1],
        z=space_point_cloud[:, 2],
        mode="markers"
    )
)
fig = fig.update_scenes(
    aspectmode="data", xaxis_autorange="reversed"
).update_traces(marker={"size": 1, "color": color_values}).show()

# 3. The robustness of stereo matching

Extract the stereo matching to a separate function

In [ ]:
def stereo_match(im_left_gray: np.ndarray, im_right_gray: np.ndarray) -> None:
    plt.subplot(1, 2, 1)
    plt.title("Left")
    plt.imshow(im_left_gray, cmap="gray")
    plt.subplot(1, 2, 2)
    plt.title("Right")
    plt.imshow(im_right_gray, cmap="gray")
    plt.show()
    plt.title("Match result")
    plt.imshow(stereo.compute(im_left_gray, im_right_gray))
    plt.show()

# try on the original pair
stereo_match(
    im_left_gray=im_left_gray,
    im_right_gray=im_right_gray
)

**Side note:** Affine transforms

In [ ]:
pt = np.array([
    [2],
    [3],
    [1] # homogen coordinate
])

# scale still works
scaled_pt = np.array([
    [5, 0, 0],
    [0, 7, 0],
    [0, 0, 1],
]) @ pt
print(f"{scaled_pt=}")

# move points with matrix multiplication
moved_pt = np.array([
    [1, 0, 9],
    [0, 1, 2],
    [0, 0, 1],
]) @ pt
print(f"{moved_pt=}")

The effect of moving one of the stereo pairs vertically by a few pixels

In [ ]:
def move_im(im: np.ndarray, dx: int, dy: int) -> np.ndarray:
    im_modified = cv2.warpAffine( # affine transform each pixel
        im, np.array([
            [1, 0, dx],
            [0, 1, dy]
        ], dtype=np.float32), (im.shape[1], im.shape[0])
    )
    return im_modified

stereo_match(
    im_left_gray=im_left_gray,
    im_right_gray=move_im(
        im=im_right_gray,
        dx=0,
        dy=4,
    )
)

Why do we experience this?

Moving the right image horizontally, by a few pixels

In [ ]:
stereo_match(
    im_left_gray=im_left_gray,
    im_right_gray=move_im(
        im=im_right_gray,
        dx=4,
        dy=0,
    )
)

Why is not there any problem here?

Cropping the images

In [ ]:
stereo_match(
    im_left_gray=im_left_gray[20:-50, 40:-20],
    im_right_gray=im_right_gray[20:-50, 40:-20]
)

Why is not there any problem here?

# 4. Estimating the surface normals

## 4.1. Theory of normals

Properties of the normal vector of a surface:

* Orthogonal to the surface in its point
* Has length 1
* Points **outside** of the surface

Calculating the normal vector of a surface at point O using two additional points on the surface.


<?xml version="1.0"?>
<svg width="450" height="230" xmlns="http://www.w3.org/2000/svg" xmlns:svg="http://www.w3.org/2000/svg">
 <!-- Created with SVG-edit - https://github.com/SVG-Edit/svgedit-->

 <g class="layer">
  <title>Layer 1</title>
  <path d="m57.84,130.57l147.16,-66.57l197,87l-344.16,-20.43z" fill="#ffff00" id="svg_11" stroke="#000000" stroke-width="5" transform="matrix(1 0 0 1 0 0)"/>
  <ellipse cx="55.84" cy="129.97" fill="#0000ff" id="svg_1" rx="18" ry="18" stroke="#000000" stroke-width="5"/>
  <ellipse cx="208.43" cy="65.12" fill="#0000ff" id="svg_6" rx="18" ry="18" stroke="#000000" stroke-width="5"/>
  <ellipse cx="402.43" cy="149.12" fill="#0000ff" id="svg_7" rx="18" ry="18" stroke="#000000" stroke-width="5"/>
  <line fill="none" id="svg_12" stroke="#ff0000" stroke-width="5" transform="matrix(1 0 0 1 0 0)" x1="55.84" x2="54.84" y1="125.57" y2="24.57"/>
  <line fill="none" id="svg_13" stroke="#ff0000" stroke-width="5" x1="32.84" x2="53.84" y1="47.57" y2="25.57"/>
  <line fill="none" id="svg_14" stroke="#ff0000" stroke-width="5" x1="77.84" x2="52.84" y1="48.57" y2="25.57"/>
  <text fill="#000000" font-family="Serif" font-size="24" id="svg_15" stroke="#ff0000" stroke-width="0" style="cursor: move;" text-anchor="middle" x="28.84" xml:space="preserve" y="166.57">O</text>
  <text fill="#000000" font-family="Serif" font-size="24" id="svg_16" stroke="#ff0000" stroke-width="0" text-anchor="middle" x="207.43" xml:space="preserve" y="39.23">P1</text>
  <text fill="#000000" font-family="Serif" font-size="24" id="svg_18" stroke="#ff0000" stroke-width="0" style="cursor: move;" text-anchor="middle" x="404.44" xml:space="preserve" y="199.23">P2</text>
  <text fill="#000000" font-family="Serif" font-size="24" id="svg_19" stroke="#ff0000" stroke-width="0" style="cursor: move;" text-anchor="middle" x="22.44" xml:space="preserve" y="87.23">N</text>
  <line fill="none" id="svg_22" stroke="#ff0000" stroke-width="5" transform="matrix(1 0 0 1 0 0)" x1="76.84" x2="181.84" y1="109.57" y2="61.57"/>
  <line fill="none" id="svg_23" stroke="#ff0000" stroke-width="5" transform="matrix(1 0 0 1 0 0)" x1="78.84" x2="88.84" y1="105.57" y2="79.57"/>
  <line fill="none" id="svg_24" stroke="#ff0000" stroke-width="5" x1="112.84" x2="80.84" y1="114.57" y2="106.57"/>
  <line fill="none" id="svg_25" stroke="#ff0000" stroke-width="5" transform="matrix(1 0 0 1 0 0)" x1="359.84" x2="83.84" y1="166.57" y2="150.57"/>
  <line fill="none" id="svg_26" stroke="#ff0000" stroke-width="5" x1="87.84" x2="110.84" y1="150.57" y2="138.57"/>
  <line fill="none" id="svg_27" stroke="#ff0000" stroke-width="5" transform="matrix(1 0 0 1 0 0)" x1="85.84" x2="109.84" y1="150.57" y2="170.57"/>
  <text fill="#000000" font-family="Serif" font-size="24" id="svg_28" stroke="#ff0000" stroke-width="0" style="cursor: move;" text-anchor="middle" x="154.44" xml:space="preserve" y="63.23">a</text>
  <text fill="#000000" font-family="Serif" font-size="24" id="svg_29" stroke="#ff0000" stroke-width="0" text-anchor="middle" x="200.36" xml:space="preserve" y="184.23">b</text>
 </g>
</svg>

The calculation of the normal vector:

1. Take the cross product $C$ of $a$ and $b$

$$C=a×b=\begin{bmatrix} a_2b_3-a_3b_2 \\ a_3b_1-a_1b_3 \\ a_1b_2-a_2b_1 \end{bmatrix}$$

2. Normalize the cross product

$$ \hat C = \frac{C}{\lVert C \rVert_2} $$

3. Use the normalized cross product ($N=\hat C$) or its negate ($N=-\hat C$) to make sure that the normal vector points **outside** of the surface.

## 4.2. Code

In [ ]:
def get_surface_normals() -> np.ndarray:
    # invert the intrinsic matrix
    proj_mat_inv = np.linalg.inv(tartanair_camera)

    # define a helper function that restores a point
    # from the pixel and the corresponding depth value
    def restore_point(x_px: int, y_px: int, z_space: float) -> np.ndarray:
        # flip the y coordinate
        y_px = disparity.shape[0] - y_px

        # basically the same as exercise 2
        pt_homogen = np.array([
            [x_px*z_space],
            [y_px*z_space],
            [z_space]
        ])
        pt_space = proj_mat_inv @ pt_homogen

        return pt_space

    surface_normals = np.zeros((disparity.shape[0], disparity.shape[1], 3), dtype=np.float32)

    # do the calculation for each pixel
    # (it was faster if we implemented this with array operations
    # but also harder to read)
    for x in range(1, surface_normals.shape[1]):
        for y in range(1, surface_normals.shape[0]):
            # check if we have a valid disparity for the following points
            #   (x, y)
            #   (x-1, y)
            #   (x, y-1)
            if min(disparity[y, x], disparity[y-1, x], disparity[y, x-1]) < 0.1:
                continue

            # get the spatial points for the pixels
            #   (x, y)
            #   (x-1, y)
            #   (x, y-1)
            o = restore_point(x, y, depth[y, x])
            p1 = restore_point(x, y-1, depth[y-1, x])
            p2 = restore_point(x-1, y, depth[y, x-1])

            # calculate the vectors pointing to the restored version of (x, y)
            a = (o-p1).flatten()
            b = (o-p2).flatten()

            # compute the normal vector
            # look for a vector that is orthogonal for both diff1 and diff2
            n = np.cross(a, b)
            n = n/np.linalg.norm(n, ord=2)
            # configure the direction of the normal vector to point to the camera
            #    idea: the camera is not inside the ground
            if np.sum(n*(-o.flatten())) < 0:
                n = -n
            n = n/np.linalg.norm(n, ord=2)

            # store the normal vector
            surface_normals[y, x] = n

    return surface_normals

surface_normals = get_surface_normals()
plt.imshow(surface_normals)

Why are the normal vectors so noisy?

# 5. Sources

<details>
    <summary>[1] TartanAir-V1 dataset</summary>
    Wang, W., Zhu, D., Wang, X., Hu, Y., Qiu, Y., Wang, C., Hu, Y., Kapoor, A., & Scherer, S. (2020). TartanAir: A Dataset to Push the Limits of Visual SLAM<br><a href="https://theairlab.org/tartanair-dataset/">https://theairlab.org/tartanair-dataset/</a>
</details>